In [1]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

import reverse_geocoder as rg
import pycountry

import warnings
warnings.filterwarnings("ignore")

# TCE dataset

Note that this dataset spans from 1950 to 2015

In [2]:
tce_df = pd.read_csv('./datasets/tce/TCE-DAT_2015-exposure_1950-2015.csv',skiprows=6)
tce_df = tce_df[tce_df['year']>=1980]
tce_df.reset_index(drop=True,inplace=True)
tce_df

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets
0,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,647217.0,2.395292e+09,14151.0,4.985341e+07,0.0,0.000000e+00
1,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,511694.0,5.355011e+10,308434.0,2.668010e+10,0.0,0.000000e+00
2,1980,1980033S12161,UNNAMED,NaN,SP,multi,SLB,25.2,49.0,2918.0,1.350719e+07,0.0,0.000000e+00,0.0,0.000000e+00
3,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,1294326.0,4.797279e+10,650256.0,2.337967e+10,0.0,0.000000e+00
4,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,128330.0,6.126082e+09,0.0,0.000000e+00,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2015,2015309N14067,MEGH,NaN,NI,multi,SOM,39.0,75.8,226629.0,6.596252e+06,40110.0,1.131492e+06,0.0,0.000000e+00
3128,2015,2015309N14067,MEGH,NaN,NI,multi,YEM,61.6,119.7,59993.0,1.188943e+08,57685.0,1.099661e+08,49016.0,9.060994e+07
3129,2015,2015312N11083,UNNAMED,NaN,NI,single,IND,19.4,37.8,846968.0,4.957173e+09,0.0,0.000000e+00,0.0,0.000000e+00
3130,2015,2015344N07145,MELOR,NaN,WP,multi,PHL,65.9,128.1,51335904.0,4.594407e+11,18618400.0,1.808132e+11,4607241.0,4.602029e+10


# IBTrACS with ISO (1980-2015)

<font color='red'>
    WARNING: This is a modified version of the IBTrACS dataset with the ISO included. The information in this notebook corresponds to storms between 1980 and 2015. You may prefer to use the whole IBTrACS-ISO dataset (ask to Bego) or generate the ISO by yourself. In the latter case, keep in mind that this will take you ~ 3 days (in your lap) to accomplish it.

In [3]:
ibtracs_df = pd.read_csv('./datasets/ibtracs/IBTrACS-1980-2015-ISO.csv')

#we append an extra column with the maximum wind speed
#(in case information from two or more agencies is provided)
MAX_WIND = ibtracs_df[['WMO_WIND','USA_WIND','TOK_WIND','CMA_WIND',
                       'HKO_WIND','NEW_WIND','REU_WIND','BOM_WIND',
                       'NAD_WIND','WEL_WIND','DS8_WIND','TD6_WIND',
                       'TD5_WIND','NEU_WIND','MLC_WIND']].max(axis=1,skipna=True)
ibtracs_df['MAX_WIND'] = MAX_WIND

#WARNING: It could happen that there are no records for 'MAX_WIND' (NaN)
#you can get rid of those records running the following
#ibtracs_df= ibtracs_df[ibtracs_df['MAX_WIND'].notna()]

ibtracs_df

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,ISO3,MAX_WIND
0,1979357S10095,1980,119,SI,MM,DANITZA:WILF,1980-01-01 00:00:00,TS,-16.11,81.68,...,NaN,9,274,1980,1,1,0,0,CCK,45.0
1,1979357S10095,1980,119,SI,MM,DANITZA:WILF,1980-01-01 03:00:00,TS,-16.09,81.34,...,NaN,7,273,1980,1,1,3,0,CCK,35.0
2,1979357S10095,1980,119,SI,MM,DANITZA:WILF,1980-01-01 06:00:00,TS,-16.07,80.96,...,NaN,8,272,1980,1,1,6,0,CCK,35.0
3,1979357S10095,1980,119,SI,MM,DANITZA:WILF,1980-01-01 09:00:00,TS,-16.06,80.48,...,NaN,10,268,1980,1,1,9,0,CCK,32.0
4,1979357S10095,1980,119,SI,MM,DANITZA:WILF,1980-01-01 12:00:00,TS,-16.10,79.90,...,NaN,12,268,1980,1,1,12,0,MUS,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238568,2015364S09190,2016,112,SP,MM,ULA,2015-12-31 12:00:00,TS,-15.80,190.50,...,NaN,5,229,2015,12,31,12,0,FJI,65.0
238569,2015364S09190,2016,112,SP,MM,ULA,2015-12-31 15:00:00,TS,-15.94,190.38,...,NaN,5,211,2015,12,31,15,0,FJI,77.0
238570,2015364S09190,2016,112,SP,MM,ULA,2015-12-31 18:00:00,TS,-16.20,190.25,...,NaN,6,211,2015,12,31,18,0,FJI,90.0
238571,2015364S09190,2016,112,SP,MM,ULA,2015-12-31 21:00:00,TS,-16.49,190.03,...,NaN,8,218,2015,12,31,21,0,FJI,87.0


As we can see, IBTrACS has much more records (rows) than TCE. This is because IBTrACS contains information on the evolution of storms (the tracks, ie the position of the storm at different times). 

# Merging strategy

MAIN IDEA: In order to merge IBTrACS with TCE we can adopt the following strategy:

1. We loop over the storms in TCE by looping over the values of IBTrACS_ID in <code>tce_df</code>
2. For each IBTrACS_ID we get the affected countries (<code>coutries</code>)
3. For each affected country we apply a boolean mask in <code>ibtracs_df</code> to get the dataframe with all the records for this storm
4. In <code>ibtracs_df</code> we select the row where <code>DIST2LAND</code> is minimum. This may give us an idea on the intensity of the storm when it hits the land
5. Selected rows are appended to a list (<code>sel_rows</code>)

WARNING: There are records that are present in TCE but that are not present in IBTrACS (see below). In this case, <code>row</code> will consist of an array of NaN. <code>count</code> amounts for the number of instances where this situation happens.

In [4]:
#IBTrACS id's found in the TCE dataset
tce_id = tce_df['IBTrACS_ID'].unique()

#list of rows (in ibtracs_df) where DIST2LAND is minimum
sel_rows = []

cont = 0

#1
for ib in tce_id:
    aux_df = tce_df[tce_df['IBTrACS_ID']==ib]
    #2
    countries = aux_df['ISO3'].unique()
    #3
    for country in countries:
        aux_df = ibtracs_df[(ibtracs_df['SID']==ib)&(ibtracs_df['ISO3']==country)]
        #4
        try:
            idxmin = aux_df['DIST2LAND'].idxmin()
            row    = aux_df.loc[idxmin].values
        except:
            #See WARNING above
            row = np.empty((len(ibtracs_df.columns),))
            row[:] = np.nan
            cont+=1
            
        #5
        sel_rows.append(row)
        
#we convert sel_rows to an array        
sel_rows = np.stack(sel_rows)
print('number of records in TCE but missing in IBTrACS:{}'.format(cont))

number of records in TCE but missing in IBTrACS:372


This is an example of a storm that appears in TCE but that is missing in IBTrACS

In [5]:
tce_df[tce_df['IBTrACS_ID']=='1980081S12170']

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,34kn_assets,64kn_pop,64kn_assets,96kn_pop,96kn_assets
9,1980,1980081S12170,UNNAMED,MR198003B012,SP,multi,TON,33.9,65.8,17200.0,112433086.0,0.0,0.0,0.0,0.0


In [6]:
ibtracs_df[(ibtracs_df['SID']=='1980081S12170')&(ibtracs_df['ISO3']=='TON')]

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,ISO3,MAX_WIND


Selected rows in <code>ibtracs_df</code> are used to create a dataframe <code>to_merge_df</code>

In [7]:
to_merge_df = pd.DataFrame(data=sel_rows,columns=ibtracs_df.columns)
to_merge_df

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,ISO3,MAX_WIND
0,1980001S13173,1980,1,SP,MM,PENI,1980-01-04 06:00:00,TS,-18.6,176.79,...,NaN,15,133,1980,1,4,6,0,FJI,60
1,1980003S15137,1980,3,SP,EA,PAUL,1980-01-04 00:00:00,TS,-16.77,138.47,...,NaN,5,145,1980,1,4,0,0,AUS,30
2,1980033S12161,1980,15,SP,MM,RAE,1980-02-02 00:00:00,TS,-12.2,161,...,NaN,17,134,1980,2,2,0,0,SLB,30
3,1980068S13068,1980,21,SI,MM,LAURE,1980-03-13 06:00:00,TS,-20.29,58.29,...,NaN,11,158,1980,3,13,6,0,MUS,105
4,1980069S12161,1980,22,SP,MM,SINA,1980-03-13 00:00:00,TS,-22.61,164.17,...,NaN,20,131,1980,3,13,0,0,NCL,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2015309N14067,2015,99,NI,AS,MEGH,2015-11-09 00:00:00,TS,12.25,50.9,...,NaN,13,263,2015,11,9,0,0,SOM,85
3128,2015309N14067,2015,99,NI,AS,MEGH,2015-11-10 09:00:00,TS,13.45,46.56,...,NaN,4,0,2015,11,10,9,0,YEM,27
3129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3130,2015344N07145,2015,108,WP,MM,MELOR,2015-12-14 09:00:00,TS,12.71,124.03,...,NaN,11,279,2015,12,14,9,0,PHL,112


<code>to_merge_df</code> has the same amount of rows, as <code>tce_df</code> as expected. However (as we saw above) there are missing records in IBTrACS, so <code>to_merge_df</code> is filled with NaN in those cases (compare record 3129 in <code>to_merge_df</code> with <code>tce_df</code>).

# Merging

Finally, we simply concatenate <code>tce_df</code> and <code>to_merge_df</code>. At this point, we have retained all of the columns in both datasets. It depends on what you want to do to drop uninteresting information.

In [8]:
union_df = pd.concat([tce_df,to_merge_df],axis=1)
union_df

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,ISO3,MAX_WIND
0,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,647217.0,...,NaN,15,133,1980,1,4,6,0,FJI,60
1,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,511694.0,...,NaN,5,145,1980,1,4,0,0,AUS,30
2,1980,1980033S12161,UNNAMED,NaN,SP,multi,SLB,25.2,49.0,2918.0,...,NaN,17,134,1980,2,2,0,0,SLB,30
3,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,1294326.0,...,NaN,11,158,1980,3,13,6,0,MUS,105
4,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,128330.0,...,NaN,20,131,1980,3,13,0,0,NCL,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3127,2015,2015309N14067,MEGH,NaN,NI,multi,SOM,39.0,75.8,226629.0,...,NaN,13,263,2015,11,9,0,0,SOM,85
3128,2015,2015309N14067,MEGH,NaN,NI,multi,YEM,61.6,119.7,59993.0,...,NaN,4,0,2015,11,10,9,0,YEM,27
3129,2015,2015312N11083,UNNAMED,NaN,NI,single,IND,19.4,37.8,846968.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3130,2015,2015344N07145,MELOR,NaN,WP,multi,PHL,65.9,128.1,51335904.0,...,NaN,11,279,2015,12,14,9,0,PHL,112


Now, if we want to get the records that apper both in TCE and IBTrACS, we drop these rows (remember <code>cont</code>?)

In [9]:
intersection_df = union_df[union_df['SID'].notna()]
intersection_df

,year,IBTrACS_ID,TC_name,NatCatSERVICE_ID,genesis_basin,countries_affected,ISO3,v_land_SI,v_land_kn,34kn_pop,...,USA_SEARAD,STORM_SPD,STORM_DR,year,month,day,hour,min,ISO3,MAX_WIND
0,1980,1980001S13173,PENI,NaN,SP,single,FJI,43.5,84.5,647217.0,...,NaN,15,133,1980,1,4,6,0,FJI,60
1,1980,1980003S15137,PAUL,MR198001B002,SP,single,AUS,51.8,100.7,511694.0,...,NaN,5,145,1980,1,4,0,0,AUS,30
2,1980,1980033S12161,UNNAMED,NaN,SP,multi,SLB,25.2,49.0,2918.0,...,NaN,17,134,1980,2,2,0,0,SLB,30
3,1980,1980068S13068,LAURE,MR198003B021,SI,single,MUS,49.3,95.9,1294326.0,...,NaN,11,158,1980,3,13,6,0,MUS,105
4,1980,1980069S12161,SINA,NaN,SP,multi,NCL,21.7,42.2,128330.0,...,NaN,20,131,1980,3,13,0,0,NCL,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3126,2015,2015301N11065,CHAPALA,NaN,NI,multi,YEM,46.6,90.6,1467373.0,...,NaN,10,273,2015,11,3,3,0,YEM,60
3127,2015,2015309N14067,MEGH,NaN,NI,multi,SOM,39.0,75.8,226629.0,...,NaN,13,263,2015,11,9,0,0,SOM,85
3128,2015,2015309N14067,MEGH,NaN,NI,multi,YEM,61.6,119.7,59993.0,...,NaN,4,0,2015,11,10,9,0,YEM,27
3130,2015,2015344N07145,MELOR,NaN,WP,multi,PHL,65.9,128.1,51335904.0,...,NaN,11,279,2015,12,14,9,0,PHL,112


Want to export the results to csv file?

In [10]:
union_df.to_csv('./datasets/TCE-IBTrACS-1980-2015_union.csv',index=False)
intersection_df.to_csv('./datasets/TCE-IBTrACS-1980-2015_intersection.csv',index=False)